In [2]:
# Importing libraries 
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
%tensorflow_version 2.x
# Importing below library to build a data input pipeline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#set logger
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

TensorFlow is already loaded. Please restart the runtime to change versions.


In [3]:
#Loading the data:
URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_dir = tf.keras.utils.get_file('cats_and_dogs_filterted.zip',origin=URL,extract=True)

68608000/68606236 [==============================] - 2s 0us/step


In [4]:
zip_dir_base = os.path.dirname(zip_dir)
!find $zip_dir_base -type d -print

/root/.keras/datasets
/root/.keras/datasets/cats_and_dogs_filtered
/root/.keras/datasets/cats_and_dogs_filtered/validation
/root/.keras/datasets/cats_and_dogs_filtered/validation/cats
/root/.keras/datasets/cats_and_dogs_filtered/validation/dogs
/root/.keras/datasets/cats_and_dogs_filtered/train
/root/.keras/datasets/cats_and_dogs_filtered/train/cats
/root/.keras/datasets/cats_and_dogs_filtered/train/dogs


In [0]:
base_dir = os.path.join(os.path.dirname(zip_dir),'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')

train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures

In [6]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

print('total training cat images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)

print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training cat images: 1000
total training dog images: 1000
total validation cat images: 500
total validation dog images: 500
--
Total training images: 2000
Total validation images: 1000


In [0]:
# Setting model parameters
BATCH_SIZE = 100 # number of training examples to process before updating our model variables.
IMG_SIZE = 150 #To reshape all the images to 150x150.

In [0]:
# Data Preparation.

train_image_generator = ImageDataGenerator(rescale=1./255) # generator for training
validation_image_generator = ImageDataGenerator(rescale=1./255) # generator for validation

In [9]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_SIZE,IMG_SIZE),
                                                           class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                              directory=validation_dir,
                                                              shuffle=False,
                                                              target_size=(IMG_SIZE,IMG_SIZE),
                                                              class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [0]:
# model creation

model = tf.keras.Sequential([
         tf.keras.layers.Conv2D(32,(3,3),activation=tf.nn.relu,input_shape=(150,150,3)),
         tf.keras.layers.MaxPooling2D(2,2),

         tf.keras.layers.Conv2D(64,(3,3),activation=tf.nn.relu),
         tf.keras.layers.MaxPooling2D(2,2),

         tf.keras.layers.Conv2D(128,(3,3),activation=tf.nn.relu),
         tf.keras.layers.MaxPooling2D(2,2),

         tf.keras.layers.Conv2D(128,(3,3),activation=tf.nn.relu),
         tf.keras.layers.MaxPooling2D(2,2),

         tf.keras.layers.Flatten(),
         tf.keras.layers.Dense(512,activation=tf.nn.relu),
         tf.keras.layers.Dense(2,activation=tf.nn.softmax)
])

In [0]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)       1

In [19]:
# Fitting the model
EPOCHS=50
history = model.fit_generator(train_data_gen,
                              steps_per_epoch=int(np.ceil(total_train/float(BATCH_SIZE))),
                              epochs=EPOCHS,
                              validation_data=val_data_gen,
                              validation_steps=int(np.ceil(total_val / float(BATCH_SIZE))))

Epoch 1/50
20/20 [==============================] - 15s 773ms/step - loss: 0.6986 - acc: 0.4950 - val_loss: 0.6907 - val_acc: 0.5000
Epoch 2/50
20/20 [==============================] - 7s 365ms/step - loss: 0.6857 - acc: 0.5505 - val_loss: 0.6648 - val_acc: 0.6210
Epoch 3/50
20/20 [==============================] - 7s 367ms/step - loss: 0.6581 - acc: 0.6080 - val_loss: 0.6519 - val_acc: 0.6270
Epoch 4/50
20/20 [==============================] - 8s 385ms/step - loss: 0.6231 - acc: 0.6565 - val_loss: 0.6234 - val_acc: 0.6770
Epoch 5/50
20/20 [==============================] - 8s 379ms/step - loss: 0.5949 - acc: 0.6975 - val_loss: 0.5872 - val_acc: 0.7070
Epoch 6/50
20/20 [==============================] - 8s 384ms/step - loss: 0.5642 - acc: 0.7290 - val_loss: 0.5781 - val_acc: 0.7200
Epoch 7/50
20/20 [==============================] - 8s 395ms/step - loss: 0.5358 - acc: 0.7725 - val_loss: 0.5625 - val_acc: 0.7370
Epoch 8/50
20/20 [==============================] - 8s 385ms/step - loss: 0

In [21]:
# Visualizing the results

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('./foo.png')
plt.show()

KeyError: ignored